#MoneyGram Lend v1.2

The App is available [here](https://moneygram-lend.anvil.app)

In [1]:
# Install Uplink to run web app
!pip install anvil-uplink
#Install Yahoo Finance for Currency Conversion
!pip install yahoo_fin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Dependencies

In [2]:

# import pickle
import pandas as pd

#Server Requirements
from google.colab import drive
import anvil.server

#Currency Convert Dependencies
import yahoo_fin.stock_info as si
from datetime import datetime, timedelta

# import StandardSclaer and OneHotEncoder
from sklearn.preprocessing import StandardScaler,OneHotEncoder

# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Import Adaboost library
from sklearn.ensemble import AdaBoostClassifier

## Process The Data

In [3]:
# Import Credit Risk information to Google Colab
url = 'https://raw.githubusercontent.com/DIsaacman/Dream-Team/dataset/Resources/credit_risk_dataset.csv'

# Read CSV
df = pd.read_csv(url)

# # Review the DataFrame
# df.head()

# drop the columns we don't want to use in our models
consolidated_df = df.drop(columns=[
       'person_emp_length', 'loan_intent', 'loan_grade',
       'loan_int_rate', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length'])

# Create a list of categorical variables 
categorical_variables = list(consolidated_df.dtypes[consolidated_df.dtypes=="object"].index)

# # Display the categorical variables list
# categorical_variables

## Perform the Machine-Learning Modelling Process (AdaBoost)

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# fit and transform the categorial variable
encoded_data = enc.fit_transform(consolidated_df[categorical_variables])

# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
columns=enc.get_feature_names_out(categorical_variables))

# create a new df that does not include those columns we converted with one_hot_encoder (all numerical variables from original dataset)
numerical_variables_df = consolidated_df.drop(columns=categorical_variables)

# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat([encoded_df, numerical_variables_df], axis=1)

# Review the Dataframe
encoded_df.head()

# Separate the data into features and target
# Define feature set
y = encoded_df["loan_status"]

# Define target set
X = encoded_df.drop(columns="loan_status")

# Split the data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=100,
                         learning_rate=1)

# Train Adaboost Classifer
abc_model = abc.fit(X_train_scaled, y_train)


## Currency Conversion Logic

In [5]:
#Connect to Anvil Server 
anvil.server.connect("VZCQ27KKQZFQ4LACSW6MW2SI-RHNXEOV2BAZECHG4")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [6]:
# Convert Currencies

#Call the Anvil Server
@anvil.server.callable

def convert_currency(amount, buy, sell):
    # construct the currency pair symbol
    symbol = f"{buy}{sell}=X"
    # extract minute data of the recent 2 days
    latest_data = si.get_data(symbol, interval="1m", start_date=datetime.now() - timedelta(days=2))
    # get the latest datetime
    last_updated_datetime = latest_data.index[-1].to_pydatetime()
    # get the latest price
    latest_price = latest_data.iloc[-1].close
    # return the latest datetime with the converted amount
    print(f"Yahoo Finance USD to AUD Conversione ($ {latest_price:.3f}) Price last updated {last_updated_datetime}")
    rate = amount * latest_price
    return  rate

# # Test: Get AUD -USD Conversion Rate
# aud_convert = convert_currency(1000,"USD","EUR")

# ML Logic

In [7]:
#Call the Anvil Server
@anvil.server.callable

#Predict Loan Function
def predict_loan(home, age, income, amount):

  if amount > income:
    return "Sorry you cannot borrow more than your income"
    
  #Format home data
  person_home_ownership_MORTGAGE = 0
  person_home_ownership_OTHER = 0
  person_home_ownership_OWN = 0
  person_home_ownership_RENT = 0

  #Conditional 
  if home == "mortgage":
    person_home_ownership_MORTGAGE = 1

  if home == "other":
    person_home_ownership_OTHER = 1

  if home == "own":
    person_home_ownership_OWN = 1

  if home == "rent":
    person_home_ownership_RENT = 1

  # User Input Scaling

  user_input = [person_home_ownership_MORTGAGE, person_home_ownership_OTHER, person_home_ownership_OWN, person_home_ownership_RENT, age, income, amount]
  user_input = X_scaler.transform([user_input])

  #Call The ML model and return a Sigmoid Classification
  classification = abc_model.predict(user_input)

  #Variable Rate based on customer risk 
  #If non risky then interest is 3%
  if classification == 0:
    amount = amount * 1.03
    response = f"Your loan will be {amount:,.2f}"
    
  #If risky then interest is 4% with initial deposit
  if classification == 1:
    deposit = amount * 0.3
    amount = amount * 1.04
    response = f"Your loan will be ${amount:,.2f} and you will need to make an initial deposit of ${deposit:,.2f}"

  #Return The Data
  return response

In [8]:
# # #Test Prediction
# loaded_model.predict([[1.0,0.0,0.0,0.0,25,9600,5500]])

In [ ]:
#Keep Colab session running 
anvil.server.wait_forever()

Yahoo Finance USD to AUD Conversione ($ 0.626) Price last updated 2022-10-11 03:10:51
Yahoo Finance USD to AUD Conversione ($ 91.248) Price last updated 2022-10-11 03:11:03


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Yahoo Finance USD to AUD Conversione ($ 0.626) Price last updated 2022-10-11 03:51:21
Yahoo Finance USD to AUD Conversione ($ 0.866) Price last updated 2022-10-11 03:51:54


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
